In [ ]:
!pip install -q moviepy openai-whisper sentence-transformers transformers google-search-results timm opencv-python pillow matplotlib tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
import os, cv2, torch, timm, random, tempfile, re
import torch.nn as nn
import numpy as np
from torchvision import transforms
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
from moviepy.editor import VideoFileClip
from serpapi import GoogleSearch
from PIL import Image
import whisper
from tqdm import tqdm
import matplotlib.pyplot as plt


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FRAME_SIZE = (224, 224)
SERPAPI_KEY = "ac4caebfbaa6418c17f1240774d56fab71495fd7bff2e4f6360957af428fca66" # Replace with your SerpAPI key
MODEL_NAME = "all-mpnet-base-v2"
VIDEO_KEYWORDS = ['youtube', 'tiktok', 'vimeo', 'bilibili', 'dailymotion']

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade  = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def extract_eyes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    eyes_list = []
    for (x,y,w,h) in faces:
        roi = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi)
        for (ex,ey,ew,eh) in eyes:
            eyes_list.append(roi[ey:ey+eh, ex:ex+ew])
    return eyes_list

transform_full = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
transform_crop = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

class RegionCNN(nn.Module):
    def __init__(self,in_ch=3,out_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch,32,3,1,1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,1,1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,1,1), nn.BatchNorm2d(128), nn.ReLU(), nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(128,out_dim)
    def forward(self,x):
        if x.ndim==3: x=x.unsqueeze(0)
        f=self.net(x).view(x.size(0),-1)
        return self.fc(f)

class HybridDetector(nn.Module):
    def __init__(self, swin_name="swin_tiny_patch4_window7_224", num_classes=2):
        super().__init__()
        self.swin = timm.create_model(swin_name, pretrained=True, num_classes=0)
        swin_dim = self.swin.num_features
        self.eye_net = RegionCNN(out_dim=128)
        self.classifier = nn.Sequential(
            nn.Linear(swin_dim+128,512), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(512,num_classes)
        )
    def forward(self,frame,eye):
        s = self.swin(frame)
        e = self.eye_net(eye)
        x = torch.cat([s,e],1)
        return self.classifier(x)

In [ ]:
def detect_deepfake(video_path, model_path="best_balanced_sampler_model.pth", interval=15):
    model = HybridDetector().to(DEVICE)
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    model.eval()

    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    preds = []
    print(f"🧩 Analyzing {frame_count} frames ...")

    while True:
        ret, frame = cap.read()
        if not ret: break
        if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) % interval != 0:
            continue

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_resized = cv2.resize(frame_rgb, FRAME_SIZE)
        eyes = extract_eyes(frame_resized)
        eye_crop = eyes[0] if len(eyes)>0 else np.zeros((32,32,3), np.uint8)

        frame_tensor = transform_full(Image.fromarray(frame_resized)).unsqueeze(0).to(DEVICE)
        eye_tensor   = transform_crop(Image.fromarray(eye_crop)).unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            out = model(frame_tensor, eye_tensor)
            pred = torch.argmax(out,1).item()
        preds.append(pred)

    cap.release()
    fake_ratio = preds.count(1)/len(preds)
    verdict = "Fake" if fake_ratio > 0.5 else "Real"
    print(f"📊 Deepfake Check → {verdict} ({fake_ratio*100:.1f}% fake frames)")
    return verdict


In [ ]:
def extract_audio(video_path):
    print("🎧 Extracting audio from video ...")
    audio_path = os.path.join(tempfile.gettempdir(), "temp_audio.wav")
    clip = VideoFileClip(video_path)
    clip.audio.write_audiofile(audio_path, logger=None)
    clip.close()
    return audio_path

def transcribe_audio(audio_path):
    print("🗣 Transcribing audio ...")
    model = whisper.load_model("small")
    result = model.transcribe(audio_path)
    return result["text"].strip()

In [ ]:
def search_web(query, max_results=10):
    params = {"engine": "google", "q": query, "num": max_results, "api_key": SERPAPI_KEY, "hl": "en"}
    search = GoogleSearch(params)
    results = search.get_dict()

    data=[]
    if "organic_results" in results:
        for r in results["organic_results"]:
            title=r.get("title",""); link=r.get("link",""); snippet=r.get("snippet","")
            if not link or not snippet: continue
            if any(v in link.lower() for v in VIDEO_KEYWORDS): continue
            data.append({"title":title,"link":link,"snippet":snippet})
    return data

def fact_check(text, embedder, nli_model):
    print("🔍 Running fact check ...")
    results = search_web(text, max_results=15)
    if not results:
        print("⚠️ No search results found.")
        return "UNCLEAR"

    texts=[r["title"]+" "+r["snippet"] for r in results]
    emb_texts=embedder.encode(texts, convert_to_tensor=True, normalize_embeddings=True)
    emb_query=embedder.encode(text, convert_to_tensor=True, normalize_embeddings=True)
    sims=util.cos_sim(emb_query, emb_texts)[0].cpu().numpy()

    ranked=[(r,float(s)) for r,s in zip(results,sims) if s>0.3]
    ranked=sorted(ranked,key=lambda x:x[1],reverse=True)[:5]
    if not ranked: return "UNCLEAR"

    entail,contra=[],[]
    for r,sim in ranked:
        snippet=r["snippet"]
        labels=["entailment","contradiction","neutral"]
        outputs=nli_model(snippet,labels)
        label=outputs["labels"][0].upper()
        score=outputs["scores"][0]
        if "ENTAIL" in label: entail.append(score)
        elif "CONTRADICT" in label: contra.append(score)

    ae=np.mean(entail) if entail else 0
    ac=np.mean(contra) if contra else 0
    if ae-ac>0.15: verdict="TRUE"
    elif ac-ae>0.15: verdict="FALSE"
    else: verdict="UNCLEAR"
    print(f"✅ Fact Check Verdict → {verdict} (Entail={ae:.2f}, Contra={ac:.2f})")
    return verdict

In [ ]:
def analyze_video(video_path):
    print(f"\n🎬 Processing video: {video_path}")
    df_verdict = detect_deepfake(video_path)
    audio = extract_audio(video_path)
    transcript = transcribe_audio(audio)
    embedder = SentenceTransformer(MODEL_NAME)
    nli_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)
    fact_verdict = fact_check(transcript, embedder, nli_model)

    if df_verdict=="Fake" and fact_verdict=="FALSE":
        final="❌ Deepfake and Misleading"
    elif df_verdict=="Fake" and fact_verdict!="FALSE":
        final="⚠️ Deepfake but facts unclear/true"
    elif df_verdict=="Real" and fact_verdict=="FALSE":
        final="⚠️ Authentic video but misinformation"
    elif df_verdict=="Real" and fact_verdict=="TRUE":
        final="✅ Authentic and factually correct"
    else:
        final="⚠️ Mixed/Unclear"

    print("\n======================================================")
    print(f"🎯 FINAL VERDICT → {final}")
    print("======================================================")
    return {"deepfake": df_verdict, "fact": fact_verdict, "final": final, "transcript": transcript}


In [ ]:
video_path = "/content/news_true.mp4"  # Replace with your video
results = analyze_video(video_path)

with open("factcheck_result.txt","w",encoding="utf-8") as f:
    f.write("=== TRANSCRIPT ===\n"+results["transcript"]+"\n\n")
    f.write(f"Deepfake Verdict: {results['deepfake']}\n")
    f.write(f"Fact Verdict: {results['fact']}\n")
    f.write(f"Final Verdict: {results['final']}\n")

print("\n✅ Saved transcript and verdicts to factcheck_result.txt")



🎬 Processing video: /content/news_true.mp4
🧩 Analyzing 126 frames ...
📊 Deepfake Check → Fake (100.0% fake frames)
🎧 Extracting audio from video ...
🗣 Transcribing audio ...


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

Device set to use cpu


🔍 Running fact check ...
✅ Fact Check Verdict → UNCLEAR (Entail=0.56, Contra=0.61)

🎯 FINAL VERDICT → ⚠️ Deepfake but facts unclear/true

✅ Saved transcript and verdicts to factcheck_result.txt
